# Tesseract

In [1]:
import re
import cv2
import glob
import pytesseract
import numpy as np
import pandas as pd
from time import sleep
from selenium import webdriver

PATH = "C:/Program Files (x86)/chromedriver.exe"
pytesseract.pytesseract.tesseract_cmd=r"C:/Users/ginel/AppData/Local/Programs/Tesseract-OCR/tesseract.exe"

imagelink = "C:/Users/ginel/OneDrive/Documents/Personal Growth/Learning/GitHub and Kaggle/Google-Tesseract/Images/"

## OPERATIONS ON IMAGES

In [2]:
# DISPLAY IMAGE

link = imagelink + "example_1.jpg"

image = cv2.imread(link, 0)

cv2.imshow("Image Displayed", image)
cv2.waitKey(0)

-1

In [3]:
# RESIZE IMAGE

link = imagelink + "example_1.jpg"

image = cv2.imread(link, 0)
image = cv2.resize(image, (500, 700))

cv2.imshow("Image Resized", image)
cv2.waitKey(0)

-1

In [4]:
# CROPPED IMAGE

link = imagelink + "example_1.jpg"

image = cv2.imread(link, 0)
image = image[50:, :200]

cv2.imshow("Image Cropped", image)
cv2.waitKey(0)

-1

In [5]:
# ROTATE IMAGE

link = imagelink + "example_1.jpg"

image = cv2.imread(link, 0)
image = cv2.rotate(image, cv2.cv2.ROTATE_90_CLOCKWISE)

cv2.imshow("Image Rotated", image)
cv2.waitKey(0)

-1

In [6]:
# TRANSLATED IMAGE

link = imagelink + "example_1.jpg"

image = cv2.imread(link, 0)

height, width = image.shape[:2]

tx, ty = width / 4, height / 4

translation_matrix = np.array([[1, 0, tx],[0, 1, ty]], dtype=np.float32)

image = cv2.warpAffine(src=image, M=translation_matrix, dsize=(width, height))

cv2.imshow("Image Translated", image)
cv2.waitKey(0)

-1

## TEXT EXTRACTION

In [7]:
link = imagelink + "example_1.jpg"

image = cv2.imread(link, 0)

data = pytesseract.image_to_string(image)

print(data)

A simple image with text to demonstrate
extraction of text using python and tesseract

“Two things are infinite: the universe and human stupidity; and I'm not sure
about the universe.” - Albert Einstein



In [8]:
link = imagelink + "example_2.jpg"

image = cv2.imread(link, 0)

data = pytesseract.image_to_string(image)

print(data)

“You've gotta dance like there's nobody watching,
Love like you'll never be hurt,
Sing like there's nobody listening,
And live like it’s heaven on earth.”
— William w. Pu rkey

